#  SVM Regression (scikit-learn)

In [1]:
%%javascript
IPython.notebook.clear_all_output();

<IPython.core.display.Javascript object>

# Import Dataset
## UCI bike sharing dataset

The example is from https://www.python-course.eu/Regression_Trees.php

We will use all 731 instances as well as a subset of the original 16 features.

We use the features: {'season', 'holiday', 'weekday', 'workingday', 'wheathersit', 'cnt'} where the {'cnt'} feature serves as our target feature and represents the number of total rented bikes per day.

The first five rows of the dataset look as follows

In [2]:
import pandas as pd
dataset = pd.read_csv("day.csv", usecols=['season','holiday','weekday','workingday','weathersit','cnt'])
print(type(dataset))
dataset.head(10)

<class 'pandas.core.frame.DataFrame'>


,season,holiday,weekday,workingday,weathersit,cnt
0,1,0,6,0,2,985
1,1,0,0,0,2,801
2,1,0,1,1,1,1349
3,1,0,2,1,1,1562
4,1,0,3,1,1,1600
5,1,0,4,1,1,1606
6,1,0,5,1,2,1510
7,1,0,6,0,2,959
8,1,0,0,0,1,822
9,1,0,1,1,1,1321


In [3]:
dataset = dataset.sample(frac=1)
dataset.head()

,season,holiday,weekday,workingday,weathersit,cnt
487,2,0,3,1,1,6169
245,3,0,6,0,1,4484
421,1,0,0,0,1,3389
538,3,0,5,1,1,5823
343,4,0,6,0,1,3190


In [4]:
# 70% data for training, 30% data for testing

training_data = dataset.iloc[:int(0.7*len(dataset))]
training_data.head()

,season,holiday,weekday,workingday,weathersit,cnt
487,2,0,3,1,1,6169
245,3,0,6,0,1,4484
421,1,0,0,0,1,3389
538,3,0,5,1,1,5823
343,4,0,6,0,1,3190


In [5]:
training_data = training_data.reset_index(drop=True)
training_data.head()

,season,holiday,weekday,workingday,weathersit,cnt
0,2,0,3,1,1,6169
1,3,0,6,0,1,4484
2,1,0,0,0,1,3389
3,3,0,5,1,1,5823
4,4,0,6,0,1,3190


In [6]:
testing_data = dataset.iloc[int(0.7*len(dataset)):].reset_index(drop=True)

X_train = training_data.iloc[:, :-1]
print(X_train.shape)
X_test = testing_data.iloc[:, :-1]
print(X_test.shape)
y_train = training_data.iloc[:, -1]
print(y_train.shape)
y_test = testing_data.iloc[:, -1]
print(y_test.shape)

(511, 5)
(220, 5)
(511,)
(220,)


# sklearn.svm.SVR

# 1. Linear Kernel: 

## Training
**C : float, optional (default=1.0):** Penalty parameter C of the error term.

**epsilon : float, optional (default=0.1):** the epsilon-tube within which there is no penalty.

In [7]:
from sklearn import svm
model = svm.SVR(kernel = 'linear', C = 1, epsilon = 0.1)

# training
model.fit(X_train, y_train)

# testing
y_predict = model.predict(X_test)
n_report = 10
print(y_predict[range(n_report)].astype(int))
print(y_test.head(n_report).values)

[4354 4850 4454 4411 4426 4571 4662 4696 4375 4521]
[7641 3974 4066 4569 1606 4649 8294 3310 4579 1969]


In [8]:
import numpy as np

# compute root-mean-square error
RMSE = (y_test - y_predict)**2
RMSE = np.sum(RMSE.values)
RMSE /= len(y_test)
RMSE = np.sqrt(RMSE)
print(RMSE)

1882.4939088697786


## Grid Search for Hyperparameters

See how to use `sklearn.model_selection.GridSearchCV(.)` :-)

In [9]:
from sklearn.model_selection import GridSearchCV
import numpy as np
# create a model with some fixed hyperparameters
model = svm.SVR(kernel = 'linear')
# create a dictionary of hyperparameters (of the model) for grid search
exp = 4
vals = np.logspace(-exp, exp, 2*exp+1) # base 10
print(vals)
params = {
    'C': vals,
    'epsilon': vals
}
# grid search
gridsearch = GridSearchCV(estimator = model, param_grid = params, verbose = 1)
# set verbose = 2 if you want to see the progress of each model training
gridsearch.fit(X_train, y_train);

# print results
print("====== Scores for all models ======")
display(gridsearch.grid_scores_) # IPython.display, a better looking function than print()
print()

print("====== Parameters for the best model ======")
print(gridsearch.best_params_)
print()

print("====== Score for the best model ======")
print(gridsearch.best_score_)

[1.e-04 1.e-03 1.e-02 1.e-01 1.e+00 1.e+01 1.e+02 1.e+03 1.e+04]
Fitting 3 folds for each of 81 candidates, totalling 243 fits
====== Scores for all models ======


[Parallel(n_jobs=1)]: Done 243 out of 243 | elapsed:    1.8s finished
/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: -0.00224, std: 0.00146, params: {'C': 0.0001, 'epsilon': 0.0001},
 mean: -0.00224, std: 0.00146, params: {'C': 0.0001, 'epsilon': 0.001},
 mean: -0.00224, std: 0.00146, params: {'C': 0.0001, 'epsilon': 0.01},
 mean: -0.00224, std: 0.00146, params: {'C': 0.0001, 'epsilon': 0.1},
 mean: -0.00224, std: 0.00146, params: {'C': 0.0001, 'epsilon': 1.0},
 mean: -0.00213, std: 0.00147, params: {'C': 0.0001, 'epsilon': 10.0},
 mean: -0.00187, std: 0.00060, params: {'C': 0.0001, 'epsilon': 100.0},
 mean: -0.00724, std: 0.00403, params: {'C': 0.0001, 'epsilon': 1000.0},
 mean: -0.01639, std: 0.01680, params: {'C': 0.0001, 'epsilon': 10000.0},
 mean: -0.00217, std: 0.00146, params: {'C': 0.001, 'epsilon': 0.0001},
 mean: -0.00217, std: 0.00146, params: {'C': 0.001, 'epsilon': 0.001},
 mean: -0.00217, std: 0.00146, params: {'C': 0.001, 'epsilon': 0.01},
 mean: -0.00217, std: 0.00146, params: {'C': 0.001, 'epsilon': 0.1},
 mean: -0.00217, std: 0.00146, params: {'C': 0.001, 'epsilon': 1.0},
 me


====== Parameters for the best model ======
{'C': 100.0, 'epsilon': 1000.0}

====== Score for the best model ======
0.27794604814596025


## Put in the best parameters

In [10]:
model = gridsearch.best_estimator_

# testing
y_predict = model.predict(X_test)
n_report = 10
print(y_predict[range(n_report)].astype(int))
print(y_test.head(n_report).values)

# report error
RMSE = (y_test - y_predict)**2
RMSE = np.sum(RMSE.values)
RMSE /= len(y_test)
RMSE = np.sqrt(RMSE)
print(RMSE)

[4027 6467 2841 3154 4266 4250 4819 6071 4134 4086]
[7641 3974 4066 4569 1606 4649 8294 3310 4579 1969]
1736.1395771930754


# 2. RBF Kernel: 

## Training
**gamma : float, optional (default=’auto’):** Kernel coefficient

**C : float, optional (default=1.0):** Penalty parameter C of the error term.

**epsilon : float, optional (default=0.1):** the epsilon-tube within which there is no penalty.

In [12]:
model = svm.SVR(kernel = 'rbf')
# create a dictionary of hyperparameters (of the model) for grid search
exp = 6
vals = np.logspace(-exp, exp, 2*exp+1) # base 10
print(vals)
params = {
    'gamma': vals,
    'C': vals,
    'epsilon': vals
}
# grid search
gridsearch = GridSearchCV(estimator = model, param_grid = params, verbose = 2)
# set verbose = 2 if you want to see the progress of each model training
gridsearch.fit(X_train, y_train);

# print results
'''
print("====== Scores for all models ======")
display(gridsearch.grid_scores_) # IPython.display, a better looking function than print()
print()
'''

print("====== Parameters for the best model ======")
print(gridsearch.best_params_)
print()

print("====== Score for the best model ======")
print(gridsearch.best_score_)

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[1.e-06 1.e-05 1.e-04 1.e-03 1.e-02 1.e-01 1.e+00 1.e+01 1.e+02 1.e+03
 1.e+04 1.e+05 1.e+06]
Fitting 3 folds for each of 2197 candidates, totalling 6591 fits
[CV] C=1e-06, epsilon=1e-06, gamma=1e-06 .............................
[CV] .............. C=1e-06, epsilon=1e-06, gamma=1e-06, total=   0.0s
[CV] C=1e-06, epsilon=1e-06, gamma=1e-06 .............................
[CV] .............. C=1e-06, epsilon=1e-06, gamma=1e-06, total=   0.0s
[CV] C=1e-06, epsilon=1e-06, gamma=1e-06 .............................
[CV] .............. C=1e-06, epsilon=1e-06, gamma=1e-06, total=   0.0s
[CV] C=1e-06, epsilon=1e-06, gamma=1e-05 .............................
[CV] .............. C=1e-06, epsilon=1e-06, gamma=1e-05, total=   0.0s
[CV] C=1e-06, epsilon=1e-06, gamma=1e-05 .............................
[CV] .............. C=1e-06, epsilon=1e-06, gamma=1e-05, total=   0.0s
[CV] C=1e-06, epsilon=1e-06, gamma=1e-05 .............................
[CV] .............. C=1e-06, epsilon=1e-06, gamma=1e-05, tot

[CV] ............... C=1e-06, epsilon=1e-05, gamma=10.0, total=   0.0s
[CV] C=1e-06, epsilon=1e-05, gamma=10.0 ..............................
[CV] ............... C=1e-06, epsilon=1e-05, gamma=10.0, total=   0.0s
[CV] C=1e-06, epsilon=1e-05, gamma=100.0 .............................
[CV] .............. C=1e-06, epsilon=1e-05, gamma=100.0, total=   0.0s
[CV] C=1e-06, epsilon=1e-05, gamma=100.0 .............................
[CV] .............. C=1e-06, epsilon=1e-05, gamma=100.0, total=   0.0s
[CV] C=1e-06, epsilon=1e-05, gamma=100.0 .............................
[CV] .............. C=1e-06, epsilon=1e-05, gamma=100.0, total=   0.0s
[CV] C=1e-06, epsilon=1e-05, gamma=1000.0 ............................
[CV] ............. C=1e-06, epsilon=1e-05, gamma=1000.0, total=   0.0s
[CV] C=1e-06, epsilon=1e-05, gamma=1000.0 ............................
[CV] ............. C=1e-06, epsilon=1e-05, gamma=1000.0, total=   0.0s
[CV] C=1e-06, epsilon=1e-05, gamma=1000.0 ............................
[CV] .

[CV] .............. C=1e-06, epsilon=0.001, gamma=0.001, total=   0.0s
[CV] C=1e-06, epsilon=0.001, gamma=0.001 .............................
[CV] .............. C=1e-06, epsilon=0.001, gamma=0.001, total=   0.0s
[CV] C=1e-06, epsilon=0.001, gamma=0.01 ..............................
[CV] ............... C=1e-06, epsilon=0.001, gamma=0.01, total=   0.0s
[CV] C=1e-06, epsilon=0.001, gamma=0.01 ..............................
[CV] ............... C=1e-06, epsilon=0.001, gamma=0.01, total=   0.0s
[CV] C=1e-06, epsilon=0.001, gamma=0.01 ..............................
[CV] ............... C=1e-06, epsilon=0.001, gamma=0.01, total=   0.0s
[CV] C=1e-06, epsilon=0.001, gamma=0.1 ...............................
[CV] ................ C=1e-06, epsilon=0.001, gamma=0.1, total=   0.0s
[CV] C=1e-06, epsilon=0.001, gamma=0.1 ...............................
[CV] ................ C=1e-06, epsilon=0.001, gamma=0.1, total=   0.0s
[CV] C=1e-06, epsilon=0.001, gamma=0.1 ...............................
[CV] .

[CV] .............. C=1e-06, epsilon=0.01, gamma=1000.0, total=   0.0s
[CV] C=1e-06, epsilon=0.01, gamma=10000.0 ............................
[CV] ............. C=1e-06, epsilon=0.01, gamma=10000.0, total=   0.0s
[CV] C=1e-06, epsilon=0.01, gamma=10000.0 ............................
[CV] ............. C=1e-06, epsilon=0.01, gamma=10000.0, total=   0.0s
[CV] C=1e-06, epsilon=0.01, gamma=10000.0 ............................
[CV] ............. C=1e-06, epsilon=0.01, gamma=10000.0, total=   0.0s
[CV] C=1e-06, epsilon=0.01, gamma=100000.0 ...........................
[CV] ............ C=1e-06, epsilon=0.01, gamma=100000.0, total=   0.0s
[CV] C=1e-06, epsilon=0.01, gamma=100000.0 ...........................
[CV] ............ C=1e-06, epsilon=0.01, gamma=100000.0, total=   0.0s
[CV] C=1e-06, epsilon=0.01, gamma=100000.0 ...........................
[CV] ............ C=1e-06, epsilon=0.01, gamma=100000.0, total=   0.0s
[CV] C=1e-06, epsilon=0.01, gamma=1000000.0 ..........................
[CV] .

...

[CV] .......... C=1000000.0, epsilon=1.0, gamma=10000.0, total=   0.0s
[CV] C=1000000.0, epsilon=1.0, gamma=100000.0 ........................
[CV] ......... C=1000000.0, epsilon=1.0, gamma=100000.0, total=   0.0s
[CV] C=1000000.0, epsilon=1.0, gamma=100000.0 ........................
[CV] ......... C=1000000.0, epsilon=1.0, gamma=100000.0, total=   0.0s
[CV] C=1000000.0, epsilon=1.0, gamma=100000.0 ........................
[CV] ......... C=1000000.0, epsilon=1.0, gamma=100000.0, total=   0.0s
[CV] C=1000000.0, epsilon=1.0, gamma=1000000.0 .......................
[CV] ........ C=1000000.0, epsilon=1.0, gamma=1000000.0, total=   0.0s
[CV] C=1000000.0, epsilon=1.0, gamma=1000000.0 .......................
[CV] ........ C=1000000.0, epsilon=1.0, gamma=1000000.0, total=   0.0s
[CV] C=1000000.0, epsilon=1.0, gamma=1000000.0 .......................
[CV] ........ C=1000000.0, epsilon=1.0, gamma=1000000.0, total=   0.0s
[CV] C=1000000.0, epsilon=10.0, gamma=1e-06 ..........................
[CV] .

[CV] ........... C=1000000.0, epsilon=100.0, gamma=0.01, total=   0.0s
[CV] C=1000000.0, epsilon=100.0, gamma=0.01 ..........................
[CV] ........... C=1000000.0, epsilon=100.0, gamma=0.01, total=   0.0s
[CV] C=1000000.0, epsilon=100.0, gamma=0.1 ...........................
[CV] ............ C=1000000.0, epsilon=100.0, gamma=0.1, total=   0.3s
[CV] C=1000000.0, epsilon=100.0, gamma=0.1 ...........................
[CV] ............ C=1000000.0, epsilon=100.0, gamma=0.1, total=   0.2s
[CV] C=1000000.0, epsilon=100.0, gamma=0.1 ...........................
[CV] ............ C=1000000.0, epsilon=100.0, gamma=0.1, total=   0.2s
[CV] C=1000000.0, epsilon=100.0, gamma=1.0 ...........................
[CV] ............ C=1000000.0, epsilon=100.0, gamma=1.0, total=   0.0s
[CV] C=1000000.0, epsilon=100.0, gamma=1.0 ...........................
[CV] ............ C=1000000.0, epsilon=100.0, gamma=1.0, total=   0.0s
[CV] C=1000000.0, epsilon=100.0, gamma=1.0 ...........................
[CV] .

[CV] ....... C=1000000.0, epsilon=100000.0, gamma=1e-05, total=   0.0s
[CV] C=1000000.0, epsilon=100000.0, gamma=0.0001 .....................
[CV] ...... C=1000000.0, epsilon=100000.0, gamma=0.0001, total=   0.0s
[CV] C=1000000.0, epsilon=100000.0, gamma=0.0001 .....................
[CV] ...... C=1000000.0, epsilon=100000.0, gamma=0.0001, total=   0.0s
[CV] C=1000000.0, epsilon=100000.0, gamma=0.0001 .....................
[CV] ...... C=1000000.0, epsilon=100000.0, gamma=0.0001, total=   0.0s
[CV] C=1000000.0, epsilon=100000.0, gamma=0.001 ......................
[CV] ....... C=1000000.0, epsilon=100000.0, gamma=0.001, total=   0.0s
[CV] C=1000000.0, epsilon=100000.0, gamma=0.001 ......................
[CV] ....... C=1000000.0, epsilon=100000.0, gamma=0.001, total=   0.0s
[CV] C=1000000.0, epsilon=100000.0, gamma=0.001 ......................
[CV] ....... C=1000000.0, epsilon=100000.0, gamma=0.001, total=   0.0s
[CV] C=1000000.0, epsilon=100000.0, gamma=0.01 .......................
[CV] .

====== Parameters for the best model ======
{'C': 100000.0, 'epsilon': 1000.0, 'gamma': 0.01}

====== Score for the best model ======
0.4172405061964149


[Parallel(n_jobs=1)]: Done 6591 out of 6591 | elapsed:  1.1min finished


In [13]:
model = gridsearch.best_estimator_

# testing
y_predict = model.predict(X_test)
n_report = 10
print(y_predict[range(n_report)].astype(int))
print(y_test.head(n_report).values)

# report error
RMSE = (y_test - y_predict)**2
RMSE = np.sum(RMSE.values)
RMSE /= len(y_test)
RMSE = np.sqrt(RMSE)
print(RMSE)

[4516 5529 2512 2510 3464 4055 5495 5186 3337 3539]
[7641 3974 4066 4569 1606 4649 8294 3310 4579 1969]
1509.2995973072532


THE END

#### In reality, you might want to normalize your data first (e.g., using sklearn.preprocessing.scale), and SVM will run faster